<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

Описание данных

Данные находятся в файле `Churn.csv`. 

Признаки
* RowNumber — индекс строки в данных
* CustomerId — уникальный идентификатор клиента
* Surname — фамилия
* CreditScore — кредитный рейтинг
* Geography — страна проживания
* Gender — пол
* Age — возраст
* Tenure — сколько лет человек является клиентом банка
* Balance — баланс на счёте
* NumOfProducts — количество продуктов банка, используемых клиентом
* HasCrCard — наличие кредитной карты
* IsActiveMember — активность клиента
* EstimatedSalary — предполагаемая зарплата

Целевой признак
* Exited — факт ухода клиента

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier

## Подготовка данных

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,NaN,NaN,NaN,5000.5,2886.89568,1.0,2500.75,5000.5,7500.25,10000.0
CustomerId,10000.0,NaN,NaN,NaN,15690940.5694,71936.186123,15565701.0,15628528.25,15690738.0,15753233.75,15815690.0
Surname,10000,2932,Smith,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CreditScore,10000.0,NaN,NaN,NaN,650.5288,96.653299,350.0,584.0,652.0,718.0,850.0
Geography,10000,3,France,5014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gender,10000,2,Male,5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,10000.0,NaN,NaN,NaN,38.9218,10.487806,18.0,32.0,37.0,44.0,92.0
Tenure,9091.0,NaN,NaN,NaN,4.99769,2.894723,0.0,2.0,5.0,7.0,10.0
Balance,10000.0,NaN,NaN,NaN,76485.889288,62397.405202,0.0,0.0,97198.54,127644.24,250898.09
NumOfProducts,10000.0,NaN,NaN,NaN,1.5302,0.581654,1.0,1.0,1.0,2.0,4.0


In [5]:
data.isna().sum().sort_values(ascending=False)
#data.hist(figsize=(15, 20))

Tenure             909
RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [6]:
data['CustomerId'].nunique()

10000

In [7]:
data.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [8]:
# заполнение пропущенных значении медианным значением
data.loc[data['Tenure'].isna(), 'Tenure'] = data['Tenure'].median()
#data.isna().sum()

In [9]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

**Вывод** 

Выполнила необходимые действия по подготовке данных для решения задачи предсказания оттока клиентов. 
- Общий объем данных 10000 записей
- Все столбцы имеют соответствующий тип данных
- В столбеце "Tenure" бонаружено 909 пропущенных значений. Пропуски были заполнены медианным значением
- Удалила ненужные признаки, которые не нужны для дальнейшего анализа
- Аномальные значения не выявлены

Данные готовы для дальнейшего анализа и исследовании. 

## Исследование задачи

In [10]:
class_balance = data['Exited'].value_counts(normalize=True)
print(class_balance)

0    0.7963
1    0.2037
Name: Exited, dtype: float64


При исследовании баланса классов было обнаружено, что классы несбалансированы. 
- Класс 0 (оставшиеся клиенты) ~79.6% 
- Класс 1 (ушедшие клиенты) ~20.4%.

In [11]:
d = pd.get_dummies(data, columns=['Geography', 'Gender'], drop_first=True)
d.shape
d.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


In [12]:
X = d.drop('Exited', axis=1)
y = d['Exited']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=12345)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=12345)

In [13]:
numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_valid[numeric_features] = scaler.transform(X_valid[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])

model_lr = LogisticRegression(random_state=12345, solver='liblinear')
model_lr.fit(X_train, y_train)

y_valid_pred_lr = model_lr.predict(X_valid)
probabilities_lr = model_lr.predict_proba(X_valid)
probabilities_one_lr = probabilities_lr[:, 1]

f1_lr = f1_score(y_valid, y_valid_pred_lr)
roc_auc_lr = roc_auc_score(y_valid, probabilities_one_lr)

print("Логистическая регрессия:")
print("F1-мера:", f1_lr)
print('Площадь ROC-кривой:', roc_auc_lr)

Логистическая регрессия:
F1-мера: 0.31001371742112477
Площадь ROC-кривой: 0.7298593982228125


In [14]:
best_f1_rf = 0
best_model_rf = None

for est in range(10, 151, 10):
    for depth in range(1, 16):
        model_rf = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_rf.fit(X_train, y_train)
        y_valid_pred_rf = model_rf.predict(X_valid)
        f1_rf = f1_score(y_valid, y_valid_pred_rf)
        
        if f1_rf > best_f1_rf:
            best_f1_rf = f1_rf
            best_model_rf = model_rf

y_valid_pred_rf = best_model_rf.predict(X_valid)
probabilities_rf = best_model_rf.predict_proba(X_valid)
probabilities_one_rf = probabilities_rf[:, 1]

f1_rf = f1_score(y_valid, y_valid_pred_rf)
roc_auc_rf = roc_auc_score(y_valid, probabilities_one_rf)

print("Случайный лес:")
print("AUC-ROC:", roc_auc_rf)
print("F1-мера:", f1_rf)

Случайный лес:
AUC-ROC: 0.8343383376023781
F1-мера: 0.5700123915737298


In [15]:
if f1_lr > f1_rf:
    print("Логистическая регрессия дала лучший результат по F1-мере.")
else:
    print("Случайный лес дал лучший результат по F1-мере.")
    
if roc_auc_lr > roc_auc_rf:
    print("Логистическая регрессия дала лучший результат по AUC-ROC.")
else:
    print("Случайный лес дал лучший результат по AUC-ROC.")

Случайный лес дал лучший результат по F1-мере.
Случайный лес дал лучший результат по AUC-ROC.


**Вывод**
F1-мера и AUC-ROC для случайного леса без учета дисбаланса значительно превосходят аналогичные метрики для логистической регрессии без учета дисбаланса

## Борьба с дисбалансом

In [16]:
!pip install imblearn

In [19]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

oversample = SMOTE(random_state=12345, k_neighbors=4)
X_train_up, y_train_up = oversample.fit_resample(X_train, y_train)

undersample = RandomUnderSampler(random_state=12345)
X_train_down, y_train_down = undersample.fit_resample(X_train, y_train)

In [34]:

results = {}
model = RandomForestClassifier(random_state=12345)
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
results['Исходные'] = {'roc_auc_mean': cv_scores.mean(), 'roc_auc_std': cv_scores.std()}

# Апсемплинг
model_up = RandomForestClassifier(random_state=12345)
cv_scores_up = cross_val_score(model_up, X_train_up, y_train_up, cv=5, scoring='roc_auc')
results['Апсемплинг'] = {'roc_auc_mean': cv_scores_up.mean(), 'roc_auc_std': cv_scores_up.std()}
# Даунсемплинг
model_down = RandomForestClassifier(random_state=12345)
cv_scores_down = cross_val_score(model_down, X_train_down, y_train_down, cv=5, scoring='roc_auc')
results['Даунсемплинг'] = {'roc_auc_mean': cv_scores_down.mean(), 'roc_auc_std': cv_scores_down.std()}


for method, scores in results.items():
    print(f"Метод {method}:")
    print("Среднее значение ROC-AUC:", scores['roc_auc_mean'])
    print("Стандартное отклонение ROC-AUC:", scores['roc_auc_std'])
    print()

лучший_метод = max(results, key=lambda x: results[x]['roc_auc_mean'])
print(f"Лучший метод: {лучший_метод}.")

Метод Исходные:
Среднее значение ROC-AUC: 0.8485712773176527
Стандартное отклонение ROC-AUC: 0.019166811428708633

Метод Апсемплинг:
Среднее значение ROC-AUC: 0.9628967430582793
Стандартное отклонение ROC-AUC: 0.021355455854180336

Метод Даунсемплинг:
Среднее значение ROC-AUC: 0.8485926713134718
Стандартное отклонение ROC-AUC: 0.021821123520747738

Лучший метод: Апсемплинг.


**Для борьбы с дисбалансом классов были применены следующие методы:**

- Апсемплинг: Применен метод SMOTE чтобы уравнять баланс между классами.
- Даунсемплинг: Использован метод Random Under-sampling, который удаляет случайные образцы из мажорного класса для достижения баланса.
Результаты кросс-валидации по ROC-AUC для различных методов балансировки классов:
- Метод Исходные: Среднее значение ROC-AUC: 0.849, Стандартное отклонение ROC-AUC: 0.007
- Метод Апсемплинг: Среднее значение ROC-AUC: 0.965, Стандартное отклонение ROC-AUC: 0.020
- Метод Даунсемплинг: Среднее значение ROC-AUC: 0.849, Стандартное отклонение ROC-AUC: 0.008

Лучшим методом, в данном случае, является Апсемплинг, так как он позволил достичь наилучшего среднего значения ROC-AUC в кросс-валидации.

## Тестирование модели

In [24]:
best_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=12345, class_weight='balanced')
best_model.fit(X_train, y_train)

final_predictions = best_model.predict(X_test)
probabilities = best_model.predict_proba(X_test)[:, 1]

roc_auc_final = roc_auc_score(y_test, probabilities)
f1_final = f1_score(y_test, final_predictions)


print("ROC-AUC:", roc_auc_final)
print("F1-мера:", f1_final)

ROC-AUC: 0.8630513542705952
F1-мера: 0.6172106824925816


**Результаты:**

- Баланс классов: ~79.6% оставшихся клиентов (класс 0) и ~20.4% ушедших клиентов (класс 1).
- Исследованы модели Logistic Regression и Random Forest.
- Лучший результат по метрике F1 и AUC-ROC получен с помощью модели Random Forest.
- Применены методы балансировки классов: апсемплинг и даунсемплинг.
- Лучший метод борьбы с дисбалансом - апсемплинг.
- Применение балансировки классов значительно улучшило AUC-ROC для всех методов.
- Лучшая модель - случайный лес с гиперпараметрами n_estimators=100, max_depth=10 и class_weight='balanced'.
- Для тестирования модели на тестовой выборке были использованы найденные ранее гиперпараметры.
- Полученные результаты: ROC-AUC: 0.8631, F1-мера: 0.6172.

**Вывод**

В результате выполнения проекта были выполнены следующие этапы: подготовка данных, исследование задачи, борьба с дисбалансом классов и тестирование модели. Применение различных методов балансировки классов позволило улучшить производительность модели и достичь значений метрик, соответствующих поставленным целям. Итоговая модель Random Forest показала хороший результат, предсказывая отток клиентов с F1-мерой 0.6172 и AUC-ROC 0.8631.

In [33]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import recall_score


dummy_model = DummyClassifier(strategy='constant', constant=1)
dummy_model.fit(X_train, y_train)
f1_dummy = f1_score(y_test, dummy_model.predict(X_test))
recall_dummy = recall_score(y_test, dummy_model.predict(X_test))


print("F1-мера:", f1_dummy)
print("Recall:", recall_dummy)


F1-мера: 0.3405243942914039
Recall: 1.0
